In [1]:
#instalo/actualizo
#! pip3 install --upgrade google-cloud-aiplatform


In [1]:
import re
import pandas as pd
from google.cloud import bigquery
#import vertexai 
#from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part, Content
#import vertexai.preview.generative_models as generative_models 
import sys
import os
from fuzzywuzzy import process, fuzz

In [2]:
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\flori\\OneDrive\\Desktop\\LDP\\ldp-data-genai-producto-b871ebffc681.json"

In [3]:
# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project='ldp-data-wyscout')


IDS

In [7]:
query = """select * from ldp-data-wyscout.transfer_market.players_bio_curated
limit 100;"""

try:
    # Ejecutar la consulta y obtener los resultados en un DataFrame                     ###domestic_competition_name
    df5 = client.query(query).to_dataframe()
    print(df5.head())
except Exception as e:
    print(f"An error occurred: {e}")



  row_ind2       row_ind       player_name name_in_home_country date_of_birth  \
0        1            NA                NA                   NA            NA   
1        1  Missing Data           A. Suhr         Missing Data            NA   
2        1  Missing Data  Christian Krämer         Missing Data            NA   
3        1  Missing Data          B. Sahin         Missing Data            NA   
4        1  Missing Data  D. Fotso Youmssi         Missing Data            NA   

  place_of_birth age        height citizenship    position  ...       tik_tok  \
0             NA  NA            NA          NA          NA  ...            NA   
1   Missing Data  16  Missing Data         N/A      Attack  ...  Missing Data   
2   Missing Data  35  Missing Data         N/A  Goalkeeper  ...  Missing Data   
3   Missing Data  16  Missing Data         N/A    midfield  ...  Missing Data   
4   Missing Data  17  Missing Data         N/A    midfield  ...  Missing Data   

   on_loan_from contract_t

In [8]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 36 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   row_ind2                         100 non-null    object
 1   row_ind                          100 non-null    object
 2   player_name                      100 non-null    object
 3   name_in_home_country             100 non-null    object
 4   date_of_birth                    100 non-null    object
 5   place_of_birth                   100 non-null    object
 6   age                              100 non-null    object
 7   height                           100 non-null    object
 8   citizenship                      100 non-null    object
 9   position                         100 non-null    object
 10  foot                             100 non-null    object
 11  player_agent                     100 non-null    object
 12  current_club                     100 

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

# Configura el controlador de Selenium
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Ejecuta el navegador en modo headless
driver_path = 'C:/chromedriver/chromedriver-win64/chromedriver.exe'  # Cambia por tu ruta del ChromeDriver

# Inicializa el controlador usando Service
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

# Función para extraer el nombre completo desde la URL
def obtener_nombre_completo(url):
    try:
        # Abre la página del jugador
        driver.get(url)
        time.sleep(2)  # Espera para asegurarte de que la página cargue completamente
        
        # Extrae el nombre completo del elemento deseado utilizando el nuevo selector
        nombre_elemento = driver.find_element(By.CSS_SELECTOR, '#tm-main > header > div.data-header__headline-container > h1')
        nombre_completo = nombre_elemento.text.strip()
        return nombre_completo
    except Exception as e:
        print(f"Error al procesar {url}: {e}")
        return None

# Aplica la función a cada fila de la columna 'player_url' y guarda los resultados en la columna 'nombre_completo'
df5['nombre_completo'] = df5['player_url'].apply(obtener_nombre_completo)

# Cierra el navegador
driver.quit()

# Verifica los primeros resultados
print(df5[['player_url', 'nombre_completo']].head())


Error al procesar NA: Message: invalid argument
  (Session info: chrome=129.0.6668.72)
Stacktrace:
	GetHandleVerifier [0x00007FF7E790B5D2+29090]
	(No symbol) [0x00007FF7E787E689]
	(No symbol) [0x00007FF7E773AFF9]
	(No symbol) [0x00007FF7E7723457]
	(No symbol) [0x00007FF7E7721A2F]
	(No symbol) [0x00007FF7E772214C]
	(No symbol) [0x00007FF7E773E231]
	(No symbol) [0x00007FF7E77D73FE]
	(No symbol) [0x00007FF7E77B66EA]
	(No symbol) [0x00007FF7E77D65D9]
	(No symbol) [0x00007FF7E77B6493]
	(No symbol) [0x00007FF7E77809B1]
	(No symbol) [0x00007FF7E7781B11]
	GetHandleVerifier [0x00007FF7E7C28C5D+3295277]
	GetHandleVerifier [0x00007FF7E7C74843+3605523]
	GetHandleVerifier [0x00007FF7E7C6A707+3564247]
	GetHandleVerifier [0x00007FF7E79C6EB6+797318]
	(No symbol) [0x00007FF7E788980F]
	(No symbol) [0x00007FF7E78853F4]
	(No symbol) [0x00007FF7E7885580]
	(No symbol) [0x00007FF7E7874A1F]
	BaseThreadInitThunk [0x00007FFF48C0257D+29]
	RtlUserThreadStart [0x00007FFF497EAF28+40]



KeyboardInterrupt: 

In [9]:
import requests
from bs4 import BeautifulSoup

def get_complete_player_name(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            # Aquí debes ajustar el selector para encontrar el nombre del jugador en la página
            player_name_tag = soup.find('h1', class_='player-name')  # Ajusta esto según la estructura de la página
            if player_name_tag:
                return player_name_tag.text.strip()
        return None
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

# Aplicar la función a cada fila del DataFrame
df5['player_name_complete'] = df5['player_url'].apply(get_complete_player_name)

# Verifica cómo se ve el DataFrame con la nueva columna
print(df5[['player_name', 'player_name_complete']].head())

Error fetching NA: Invalid URL 'NA': No scheme supplied. Perhaps you meant https://NA?
        player_name player_name_complete
0                NA                 None
1           A. Suhr                 None
2  Christian Krämer                 None
3          B. Sahin                 None
4  D. Fotso Youmssi                 None


In [10]:
df5.head()

,row_ind2,row_ind,player_name,name_in_home_country,date_of_birth,place_of_birth,age,height,citizenship,position,...,on_loan_from,contract_there_expires,full_name,contract_option,x2nd_club,timestamp,player_url,blob,player_id,player_name_complete
0,1,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,None,None,None
1,1,Missing Data,A. Suhr,Missing Data,NA,Missing Data,16,Missing Data,N/A,Attack,...,Missing Data,Missing Data,Missing Data,Missing Data,Missing Data,2023-09-21 02:47:45,https://www.transfermarkt.com/a-suhr/profil/sp...,None,1043726,None
2,1,Missing Data,Christian Krämer,Missing Data,NA,Missing Data,35,Missing Data,N/A,Goalkeeper,...,Missing Data,Missing Data,Missing Data,Missing Data,Missing Data,2023-09-21 02:47:45,https://www.transfermarkt.com/christian-kramer...,None,423055,None
3,1,Missing Data,B. Sahin,Missing Data,NA,Missing Data,16,Missing Data,N/A,midfield,...,Missing Data,Missing Data,Missing Data,Missing Data,Missing Data,2023-09-21 02:47:45,https://www.transfermarkt.com/b-sahin/profil/s...,None,1033835,None
4,1,Missing Data,D. Fotso Youmssi,Missing Data,NA,Missing Data,17,Missing Data,N/A,midfield,...,Missing Data,Missing Data,Missing Data,Missing Data,Missing Data,2023-09-21 02:47:45,https://www.transfermarkt.com/d-fotso-youmssi/...,None,1043562,None


DOCU FUZZY  https://pypi.org/project/fuzzywuzzy/

In [53]:
fuzz.ratio("this is a test", "this is a test!")

97

In [54]:
fuzz.partial_ratio("this is a test", "this is a test!")
 

100

In [55]:
fuzz.token_sort_ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")

100

In [57]:
fuzz.token_sort_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

84

In [58]:
fuzz.token_set_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

100

UNION DE DF CON LOS PAISES EN INGLES DE LA TABLA SALARIOS_COPY CON DICT TEAMS USANDO FUZZY----


In [37]:
# Normalizar los nombres en ambas tablas
df1['club'] = df1['club'].str.lower().str.strip()
df1['liga'] = df1['liga'].str.lower().str.strip()
df1['pais'] = df1['pais'].str.lower().str.strip()
df1['region'] = df1['region'].str.lower().str.strip()

df2['name'] = df2['name'].str.lower().str.strip()
df2['officialName'] = df2['officialName'].str.lower().str.strip()
df2['domestic_competition_name'] = df2['domestic_competition_name'].str.lower().str.strip()
df2['area_name'] = df2['area_name'].str.lower().str.strip()
df2['region'] = df2['region'].str.lower().str.strip()

# Función para encontrar la mejor coincidencia en una lista
def get_best_match(value, choices):
    match, score = process.extractOne(value, choices, scorer=fuzz.token_sort_ratio)
    return match if score >= 50 else None

# Encontrar el ID de los clubes en la segunda tabla
df1['wyId'] = df1['club'].apply(lambda x: get_best_match(x, df2['name'].tolist()))

# Mapear los nombres encontrados a los IDs reales en df2
name_to_wyId = dict(zip(df2['name'], df2['wyId']))
df1['wyId'] = df1['wyId'].map(name_to_wyId)

# Merge basado en la columna 'wyId' para obtener la información completa
merged_df = pd.merge(df1, df2, left_on='wyId', right_on='wyId', how='inner', suffixes=('_tabla1', '_tabla2'))

# Verifica cómo se ve el merge
print(merged_df[['club', 'wyId', 'name', 'officialName']].head(10))

# Filtrar columnas y organizar el DataFrame final
final_columns2 = [
    'region_tabla1', 'pais', 'liga', 'club', 
    'weeklyavg', 'annualavg', 'moneda', 'wyId', 'name', 
    'officialName', 'area_name', 'domestic_competition_name', 'region_tabla2'
]

# Asegúrate de que las columnas existen en el DataFrame
missing_columns = [col for col in final_columns2 if col not in merged_df.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")

# Seleccionar solo las columnas presentes en merged_df
final_columns2_existing = [col for col in final_columns2 if col in merged_df.columns]
final_df2 = merged_df[final_columns2_existing]

# Renombrar columnas para claridad
final_df2.rename(columns={
    'region_tabla1': 'region1',
    'pais': 'pais',
    'liga': 'liga',
    'club': 'club',
    'weeklyavg': 'weeklyavg',
    'annualavg': 'annualavg',
    'moneda': 'moneda',
    'wyId': 'wyId',
    'name': 'name',
    'officialName': 'officialName',
    'area_name': 'area_name',
    'domestic_competition_name': 'domestic_competition_name',
    'region_tabla2': 'region2'
}, inplace=True)

# Guardar o imprimir el DataFrame final
final_df2.to_csv('tabla_combinada.csv', index=False)
print(final_df2)


                     club   wyId                name            officialName
0          atlanta united  58165      atlanta united       atlanta united fc
1  new england revolution   7854         new england  new england revolution
2      philadelphia union   8061  philadelphia union      philadelphia union
3        seattle sounders   8067    seattle sounders     seattle sounders fc
4            nashville sc  30149        nashville sc            nashville sc
5        minnesota united   8095    minnesota united     minnesota united fc
6            orlando city   8068        orlando city         orlando city sc
7              toronto fc  14995             toronto              toronto fc
8           fc cincinnati  37892          cincinnati           fc cincinnati
9         colorado rapids   7853     colorado rapids         colorado rapids
                   region1           pais         liga  \
0    central/north america  united states          mls   
1    central/north america  united st

C:\Users\flori\AppData\Local\Temp\ipykernel_26872\184841794.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df2.rename(columns={


In [38]:
final_df2['wyId'] = final_df2['wyId'].astype(int)


C:\Users\flori\AppData\Local\Temp\ipykernel_26872\3153981150.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df2['wyId'] = final_df2['wyId'].astype(int)


In [76]:
# Renombrar columnas duplicadas sino me da error al intentar subirlo a BQ
#final_df2.columns = [
#    'region_tabla1','pais','liga','club', 
#    'weeklyavg', 'annualavg', 'moneda', 'wyId', 'name', 
#    'officialName', 'area_name','domestic_competition_name' ,'region_tabla2'
#]


In [39]:
from google.cloud import bigquery

# Especifica tu ID de proyecto
project_id = 'ldp-data-wyscout'

# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project=project_id)

# Especificar la tabla de destino en el formato dataset.tabla
table_id = 'ldp-data-wyscout.capology.fuzzy2'

# Configurar la carga de datos
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Reemplaza la tabla si ya existe
)

# Guardar el DataFrame en BigQuery
job = client.load_table_from_dataframe(final_df2, table_id, job_config=job_config)
job.result()  # Esperar a que el trabajo termine

print(f"Los datos han sido cargados en la tabla {table_id}.")

Los datos han sido cargados en la tabla ldp-data-wyscout.capology.fuzzy2.


ahora priorizando pais

In [33]:
from fuzzywuzzy import process, fuzz
import pandas as pd

# Normalizar las columnas
df1['club'] = df1['club'].str.lower().str.strip()
df1['pais'] = df1['pais'].str.lower().str.strip()
df1['region'] = df1['region'].str.lower().str.strip()

df2['name'] = df2['name'].str.lower().str.strip()
df2['officialName'] = df2['officialName'].str.lower().str.strip()
df2['area_name'] = df2['area_name'].str.lower().str.strip()
df2['region'] = df2['region'].str.lower().str.strip()

# Función para encontrar la mejor coincidencia
def get_best_match(value, choices):
    match, score = process.extractOne(value, choices, scorer=fuzz.token_sort_ratio)
    return match if score >= 60 else None

# Inicializar una lista para almacenar los IDs
df1['wyId'] = None

# Priorizar los matchings
for column in ['club', 'pais', 'region']:
    if column == 'club':
        df1['wyId'] = df1['club'].apply(lambda x: get_best_match(x, df2['name'].tolist()))
    elif column == 'pais':
        # Solo buscar si no hay match en club
        df1.loc[df1['wyId'].isnull(), 'wyId'] = df1.loc[df1['wyId'].isnull(), 'pais'].apply(
            lambda x: get_best_match(x, df2['area_name'].tolist())
        )
    elif column == 'region':
        # Solo buscar si no hay match en pais
        df1.loc[df1['wyId'].isnull(), 'wyId'] = df1.loc[df1['wyId'].isnull(), 'region'].apply(
            lambda x: get_best_match(x, df2['region'].tolist())
        )

# Mapear los nombres encontrados a los IDs reales en df2
name_to_wyId = dict(zip(df2['name'], df2['wyId']))
df1['wyId'] = df1['wyId'].map(name_to_wyId)

# Merge basado en la columna 'wyId'
merged_df = pd.merge(df1, df2, on='wyId', how='inner', suffixes=('_tabla1', '_tabla2'))

# Verifica cómo se ve el merge
print(merged_df[['club', 'wyId', 'name', 'officialName']].head(10))

# Filtrar columnas y organizar el DataFrame final
final_columns2 = [
    'region_tabla1', 'pais', 'club', 
    'weeklyavg', 'annualavg', 'moneda', 'wyId', 'name', 
    'officialName', 'area_name', 'region_tabla2'
]

# Asegúrate de que las columnas existen en el DataFrame
missing_columns = [col for col in final_columns2 if col not in merged_df.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")

# Seleccionar solo las columnas presentes en merged_df
final_columns2_existing = [col for col in final_columns2 if col in merged_df.columns]
final_df22 = merged_df[final_columns2_existing]

# Renombrar columnas para claridad
final_df22.rename(columns={
    'region_tabla1': 'region1',
    'pais': 'pais',
    'club': 'club',
    'weeklyavg': 'weeklyavg',
    'annualavg': 'annualavg',
    'moneda': 'moneda',
    'wyId': 'wyId',
    'name': 'name',
    'officialName': 'officialName',
    'area_name': 'area_name',
    'region_tabla2': 'region2'
}, inplace=True)

# Guardar o imprimir el DataFrame final
final_df22.to_csv('tabla_combinada2.csv', index=False)
print(final_df22)


                     club     wyId                name            officialName
0          atlanta united  58165.0      atlanta united       atlanta united fc
1  new england revolution   7854.0         new england  new england revolution
2      philadelphia union   8061.0  philadelphia union      philadelphia union
3        seattle sounders   8067.0    seattle sounders     seattle sounders fc
4            nashville sc  30149.0        nashville sc            nashville sc
5        minnesota united   8095.0    minnesota united     minnesota united fc
6            orlando city   8068.0        orlando city         orlando city sc
7              toronto fc  14995.0             toronto              toronto fc
8           fc cincinnati  37892.0          cincinnati           fc cincinnati
9         colorado rapids   7853.0     colorado rapids         colorado rapids
                   region1           pais                    club  weeklyavg  \
0    central/north america  united states          

C:\Users\flori\AppData\Local\Temp\ipykernel_26872\4125227688.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df22.rename(columns={


In [34]:
from google.cloud import bigquery

# Especifica tu ID de proyecto
project_id = 'ldp-data-wyscout'

# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project=project_id)

# Especificar la tabla de destino en el formato dataset.tabla
table_id = 'ldp-data-wyscout.capology.fuzzy'

# Configurar la carga de datos
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Reemplaza la tabla si ya existe
)

# Guardar el DataFrame en BigQuery
job = client.load_table_from_dataframe(final_df2, table_id, job_config=job_config)
job.result()  # Esperar a que el trabajo termine

print(f"Los datos han sido cargados en la tabla {table_id}.")

Los datos han sido cargados en la tabla ldp-data-wyscout.capology.fuzzy.
